## Import

In [3]:
import random
import pandas as pd
import numpy as np
import os

import librosa

from tqdm.auto import tqdm

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder

import datetime as dt
from time import time

import warnings
warnings.filterwarnings(action='ignore')

###
from sklearn.neighbors import KNeighborsClassifier
###



from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join




# LSTM + sequential forward selection(SFS) 적용해보자

In [ ]:
CFG = {
    'SR':16000,     # 고정시킬거임
    'N_MFCC':32,    # MFCC 벡터를 추출할 개수 조정할지도 모름
    'SEED':41
}

'''
① sr

sampling rate를 말합니다. default값은 22050Hz입니다. 음성 데이터를 load 할 때 sr을 16000Hz으로 하면 꼭 sr=16000을 파라미터로 삽입해야 합니다. (사람의 목소리는 대부분 16000Hz 안에 포함된다고 합니다)

② n_mfcc

return 될 mfcc의 개수를 정해주는 파라미터입니다. default값은 20입니다. 더 다양한 데이터 특징을 추출하기 위해서는 100까지 증가 시키기도 합니다.

③ n_fft

frame의 length를 결정하는 파라미터 입니다. n_fft를 설정하면 window size가 자동으로 같은 값으로 설정되는데 window size의 크기로 잘린 음성이 n_fft보다 작은 경우 0으로 padding을 붙여주는 작업을 하기 때문에 n_fft는 window size보다 크거나 같아야 합니다. 

일반적으로 자연어 처리에서는 음성을 25m의 크기를 기본으로 하고 있으며 16000Hz인 음성에서는 400에 해당하는 값입니다. (16000 * 0.025 = 400) 즉, n_fft는 sr에 frame_length인 0.025를 곱한 값입니다.

④ hop_length

hop_length의 길이만큼 옆으로 가면서 데이터를 읽습니다. 10ms를 기본으로 하고 있어 16000Hz인 음성에서는 160에 해당합니다. (16000 * 0.01 = 160) 즉, hop_length는 sr에 frame_stride인 0.01를 곱해서 구할 수 있습니다.

window_length가 0.025이고 frame_stride가 0.01이라고 하면 0.015초씩은 데이터를 겹치면서 읽는다고 생각하면 됩니다.

내용 출처 : https://youdaeng-com.tistory.com/5

'''

## Fixed Random-Seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing 1

In [ ]:
train_df = pd.read_csv('./train_data.csv')
test_df = pd.read_csv('./test_data.csv')
print(train_df.shape, test_df.shape)

In [ ]:
train_df.head(30)

#### 30개 파일들을 직접 듣고 대략적으로 어떤 느낌인지 들어보자
#### . . .
#### 전혀 모르겠다. 안 걸린 녀석이 걸린 녀석처럼 기침한다...
#### 이게 가능한 주제일까?

In [ ]:
test_df.head()

In [ ]:
train_df[train_df['covid19']==1]

In [ ]:
train_df[train_df['covid19']==1].shape

In [ ]:
def get_mfcc_feature(df, data_type, save_path):
    # Data Folder path
    root_folder = './wav_dataset' 
    if os.path.exists(save_path):
        print(f'{save_path} is exist.')
        return
    features = []
    for uid in tqdm(df['id']):
        root_path = os.path.join(root_folder, data_type)
        path = os.path.join(root_path, str(uid).zfill(5)+'.wav')

        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
                                                                                '''
                                                                                MFCC(Mel-Frequency Cepstral Coefficient)
                                                                                - 멜 스펙트럼에서 켑스트럴 분석을 통해 추출된 값이다.
                                                                                    - Cepstral은 스펙트럼에서 배음 구조를 유추할 수 있도록 도와주는 분석이다.
                                                                                - 로그 멜 스펙트럼에 역푸리에변환(Inverse Fourier Transform)을 적용한다.
                                                                                    - 주파수 정보의 상관 관계가 높은 문제를 해소한다.
                                                                                '''
        
        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)
    
    # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    df = pd.concat([df, mfcc_df], axis=1)
    df.to_csv(save_path, index=False)
    print('Done.')

In [ ]:
get_mfcc_feature(train_df, 'train', './train_mfcc_data.csv')
get_mfcc_feature(test_df, 'test', './test_mfcc_data.csv')

In [ ]:
test_df.head()

## Data Pre-Processing 2

In [ ]:
# wav 파일의 MFCC Feature와 상태정보를 합친 학습데이터를 불러옵니다.
train_df = pd.read_csv('./train_mfcc_data.csv')

# 학습데이터를 모델의 input으로 들어갈 x와 label로 사용할 y로 분할
train_x = train_df.drop(columns=['id', 'covid19'])
train_y = train_df['covid19']

In [ ]:
def onehot_encoding(ohe, x):
    # 학습데이터로 부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
    encoded = ohe.transform(x['gender'].values.reshape(-1,1))
    encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
    x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
    return x

In [ ]:
# 'gender' column의 경우 추가 전처리가 필요 -> OneHotEncoder 적용
ohe = OneHotEncoder(sparse=False)
ohe.fit(train_x['gender'].values.reshape(-1,1))
train_x = onehot_encoding(ohe, train_x)

## Train

In [ ]:
model = MLPClassifier(random_state=CFG['SEED']) # Sklearn에서 제공하는 Multi-layer Perceptron classifier 사용
model.fit(train_x, train_y) # Model Train

## Inference

In [ ]:
# 위의 학습데이터를 전처리한 과정과 동일하게 test data에도 적용
test_x = pd.read_csv('./test_mfcc_data.csv')
test_x = test_x.drop(columns=['id'])
# Data Leakage에 유의하여 train data로만 학습된 ohe를 사용
test_x = onehot_encoding(ohe, test_x)

# Model 추론
preds = model.predict(test_x)

## Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['covid19'] = preds
submission.to_csv('submit2.csv', index=False)

In [ ]:
date_now = datetime.now().strftime('%Y%m%d_%H%M%S')

In [ ]:
import datetime

In [ ]:
datetime.timedelta(weeks=1)

In [ ]:
datetime.timedelta